# Welcome to Exkaldi

In this section, we will make a HCLG decode graph.

In [1]:
import exkaldi

import os
dataDir = os.path.join("..","examplesdata","librispeech_dummy")

Prepare lexicons (generated in 3_prepare_lexicons).

In [2]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

Like the traing of HMM and decision tree, we also have high-level API to make HCLG graph. But firstly we introduce how to build HCLG graph with exkaldi toolkit in detail.

### Make HCLG in detail

Make G file from ARPA language model (generated in 4_train_and_query_language_model).

This is a 3-grams LM.

In [4]:
outDir = os.path.join(dataDir, "exp", "graph")

exkaldi.utils.make_dependent_dirs(outDir, False)

In [5]:
arpaFile = os.path.join(dataDir, "exp", "3-gram.arpa")
Gfile = os.path.join(outDir, "G.fst")

exkaldi.decode.graph.make_G(lexicons, arpaFile, outFile=Gfile, order=3)

'/misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_dummy/exp/graph/G.fst'

Then compose __L.fst__ and __G.fst__ to __LG.fst__. 

__L.fst__ file has been generated before (in 3_prepare_lexicons), use it directly.

In [6]:
Lfile = os.path.join(dataDir, "exp", "L_disambig.fst")
LGfile = os.path.join(outDir, "LG.fst")

exkaldi.decode.graph.compose_LG(Lfile, Gfile, outFile=LGfile)

'/misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_dummy/exp/graph/LG.fst'

Then compose __LG__ and context information to __CLG.fst__. 

__tree__ will be used here. and __ilabel__ info will also be generated in this step.

In [7]:
treeFile = os.path.join(dataDir, "exp", "train_tree", "tree")
CLGfile = os.path.join(outDir, "CLG.fst")

_, ilabelFile = exkaldi.decode.graph.compose_CLG(lexicons, treeFile, LGfile, outFile=CLGfile)

ilabelFile

'/misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_dummy/exp/graph/CLG.ilabels'

Last, compose all infos to final __HCLG.fst__ graph.

In [9]:
hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")
HCLGFile = os.path.join(outDir, "HCLG.fst")

exkaldi.decode.graph.compose_HCLG(hmmFile, treeFile, CLGfile, ilabelFile, outFile=HCLGFile)

'/misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_dummy/exp/graph/HCLG.fst'

### Make HCLG with high-level API

In [8]:
os.remove(Gfile)
os.remove(LGfile)
os.remove(CLGfile)
os.remove(ilabelFile)
os.remove(HCLGFile)

In [10]:
outDir = os.path.join(dataDir, "exp", "graph")

exkaldi.decode.graph.make_graph(lexicons, arpaFile, hmmFile, treeFile, tempDir=tmpDir, useLFile=Lfile)

Skip making L. Use: ../examplesdata/librispeech_light/exp/L_disambig.fst.
Make G done: /misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_light/exp/graph/G.fst.
Compose LG done: /misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_light/exp/graph/LG.fst.
Compose CLG done: /misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_light/exp/graph/CLG.fst.
Ilabel info: /misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_light/exp/graph/CLG.ilabels.


'/misc/Work19/wangyu/exkaldi-1.0/examplesdata/librispeech_light/exp/graph/HCLG.fst'

Here we specified "useLFile" to use the L file generated before. If you don't want this, specify "useLFile" None, and other parameters, such as "useDisambigLexicons" and so on.